In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jan  9 14:14:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    28W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [16]:
################################################################################
# DATASETS
################################################################################


class Dataset:
    def __init__(self, X_train, y_train, X_test, y_test, shape, shape_flattened):
        X_train = X_train.astype(dtype) / 255.0
        y_train = y_train.astype(dtype)
        X_test = X_test.astype(dtype)  / 255.0
        y_test = y_test.astype(dtype)

        X_train = np.reshape(X_train, shape_flattened)
        X_test = np.reshape(X_test, shape_flattened)

        X = np.concatenate((X_train, X_test), axis=0)
        y = np.concatenate((y_train, y_test), axis=0)

        from sklearn.preprocessing import StandardScaler

        scaler = StandardScaler()
        scaler.fit(X_train)  # Scaling each feature independently

        X_norm = scaler.transform(X)
        X_train_norm = scaler.transform(X_train)
        X_test_norm = scaler.transform(X_test)

        X_norm = np.reshape(X_norm, shape)
        X_train_norm = np.reshape(X_train_norm, shape)
        X_test_norm = np.reshape(X_test_norm, shape)

        self.X_norm = X_norm
        self.y = y
        self.X_train_norm = X_train_norm
        self.y_train = y_train
        self.X_test_norm = X_test_norm
        self.y_test = y_test


def get_cifar_10_dataset():
    cifar10 = tf.keras.datasets.cifar10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_cifar_100_dataset():
    cifar100 = tf.keras.datasets.cifar100
    (X_train, y_train), (X_test, y_test) = cifar100.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_svhn_dataset():
    from scipy import io

    X_train = io.loadmat(train_filename, variable_names='X').get('X')
    y_train = io.loadmat(train_filename, variable_names='y').get('y')
    X_test = io.loadmat(test_filename, variable_names='X').get('X')
    y_test = io.loadmat(test_filename, variable_names='y').get('y')

    X_train = np.moveaxis(X_train, -1, 0)
    y_train -= 1
    X_test = np.moveaxis(X_test, -1, 0)
    y_test -= 1

    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_mnist_dataset():
    mnist = tf.keras.datasets.mnist
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


def get_fashion_mnist_dataset():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened)


################################################################################
# REGULARIZERS
################################################################################


class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.n_new_neurons = 0
        self.scaling_tensor = None
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
        else:
            self.update_scaling_tensor = None

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'weighted_l1_reordered':
            return self.weighted_l1_reordered(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_tensor = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)
        weighted_values = scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def weighted_l1_reordered(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        if self.update_scaling_tensor:
            scaling_tensor_raw = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)

            scaling_tensor_old_neurons = scaling_tensor_raw[:, :-self.n_new_neurons]
            scaling_tensor_new_neurons = scaling_tensor_raw[:, -self.n_new_neurons:]
            scaling_tensor_old_neurons_shuffled = tf.transpose(tf.random.shuffle(tf.transpose(scaling_tensor_old_neurons)))
            self.scaling_tensor = tf.concat([scaling_tensor_old_neurons_shuffled, scaling_tensor_new_neurons], axis=-1)
            self.update_scaling_tensor = False

        weighted_values = self.scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)
    
    def prune(self):
        self.n_new_neurons = 0
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
    
    def grow(self, n_new_neurons):
        self.n_new_neurons = n_new_neurons
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


################################################################################
# LAYERS
################################################################################


class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape):
        super().__init__()

        self.inpt_shp = input_shape


class Dense(CustomLayer):
    def __init__(self, units, activation, regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)

        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
    
    def build(self, input_shape):
        input_units = input_shape[-1]

        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, self.units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs, training=None):
        return self.A(tf.matmul(inputs, self.W) + self.b)

    def get_size(self):
        return self.W.shape[0], self.W.shape[1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_W = tf.gather(self.W.value(), active_input_units_indices, axis=0)

        if self.fixed_size:
            active_output_neurons_indices = list(range(new_W.shape[1]))
        else:
            # Prune units in this layer
            weights_with_biases = tf.concat([new_W, tf.reshape(self.b.value(), (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_output_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W = tf.gather(new_W, active_output_neurons_indices, axis=1)
            new_b = tf.gather(self.b.value(), active_output_neurons_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.prune()
        return active_output_neurons_indices
    
    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            W_growth = self.W_init(shape=(self.W.shape[0] + n_new_input_units, self.W.shape[1]), dtype=dtype)[-n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_W = tf.concat([self.W.value(), W_growth], axis=0)
        else:
            new_W = self.W.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_W.shape[1] * percentage))
            W_growth = self.W_init(shape=(new_W.shape[0], new_W.shape[1] + n_new_output_units), dtype=dtype)[:, -n_new_output_units:] * scaling_factor
            b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
            new_W = tf.concat([new_W, W_growth], axis=1)
            new_b = tf.concat([self.b.value(), b_growth], axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units
    
    def get_param_string():
        param_string = ""
        weights_with_bias = tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string


class Conv2D(CustomLayer):
    def __init__(self, filters, filter_size, activation, strides=(1, 1), 
                 padding='SAME', regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform',
                 bias_initializer='zeros', input_shape=None, fixed_size=False):
        super().__init__(input_shape)
    
        self.filters = filters
        self.filter_size = filter_size
        self.activation = activation
        self.strides = strides
        self.padding = padding
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.F_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
    
    def build(self, input_shape):
        input_filters = input_shape[-1]

        self.F = tf.Variable(
            name='F',
            initial_value=self.F_init(
                shape=(self.filter_size[0], self.filter_size[1], input_filters, self.filters), dtype=dtype
            ),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.filters,), dtype=dtype),
            trainable=True)

        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([tf.reshape(self.F, (-1, self.F.shape[-1])), tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs, training=None):
        y = tf.nn.conv2d(inputs, self.F, strides=self.strides, padding=self.padding)
        y = tf.nn.bias_add(y, self.b)
        y = self.A(y)
        return y
    
    def get_size(self):
        return self.F.shape[-2], self.F.shape[-1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_F = tf.gather(self.F.value(), active_input_units_indices, axis=-2)

        if self.fixed_size:
            active_output_filters_indices = list(range(new_F.shape[-1]))
        else:
            # Prune units in this layer
            F_reduced_max = tf.reshape(tf.math.reduce_max(tf.abs(new_F), axis=(0, 1, 2)), (1, -1))
            F_reduced_max_with_biases = tf.concat([F_reduced_max, tf.reshape(self.b.value(), (1, -1))], axis=0)
            filters_are_active = tf.math.reduce_max(tf.abs(F_reduced_max_with_biases), axis=0) >= threshold
            active_output_filters_indices = tf.reshape(tf.where(filters_are_active), (-1,))
            
            new_F = tf.gather(new_F, active_output_filters_indices, axis=-1)
            new_b = tf.gather(self.b.value(), active_output_filters_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.prune()
        return active_output_filters_indices

    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            F_growth = self.F_init(shape=(self.F.shape[0], self.F.shape[1], self.F.shape[2] + n_new_input_units, self.F.shape[3]), dtype=dtype)[:, :, -n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_F = tf.concat([self.F.value(), F_growth], axis=-2)
        else:
            new_F = self.F.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_F.shape[-1] * percentage))
            F_growth = self.F_init(shape=(new_F.shape[0], new_F.shape[1], new_F.shape[2], new_F.shape[3] + n_new_output_units), dtype=dtype)[:, :, :, -n_new_output_units:] * scaling_factor
            b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
            new_F = tf.concat([new_F, F_growth], axis=-1)
            new_b = tf.concat([self.b.value(), b_growth], axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units

    def get_param_string():
        param_string = ""
        # TODO
        return param_string


class Flatten(tf.keras.Model):
    def call(self, inputs, training=None):
        return tf.reshape(tf.transpose(inputs, perm=[0, 3, 1, 2]), (inputs.shape[0], -1))


################################################################################
# MODELS
################################################################################


class Sequential(tf.keras.Model):
    def __init__(self, layers, activation=None):
        super().__init__()
        
        self.lrs = layers
        
    def call(self, inputs, training=None):
        x = inputs
        for layer in self.lrs:
            x = layer(x, training=training)
        return x
    
    def get_layer_input_shape(self, target_layer):
        if target_layer.inpt_shp is not None:
            return target_layer.inpt_shp

        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            if layer is target_layer:
                return tuple(input.shape[1:])
            input = layer(input)
        raise Exception("Layer not found in the model.")

    def get_layer_output_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            output = layer(input)
            if layer is target_layer:
                return tuple(output.shape[1:])
            input = output
        raise Exception("Layer not found in the model.")
    
    def get_layer_sizes(self):
        """
        Returns the sizes of all layers in the model, including the input and output layer.
        """
        layer_sizes = list()
        first_layer = True
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                layer_size = layer.get_size()
                if first_layer:
                    layer_sizes.append(layer_size[0])
                    first_layer = False
                layer_sizes.append(layer_size[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def remove_regularization(self):
        self.set_regularization_penalty(0.)
    
    def get_regularization_penalty(self):
        #TODO improve
        return self.lrs[-2].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for layer in self.lrs:
            if isinstance(layer, CustomLayer) and not layer.fixed_size:
                layer.regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, params):
        input_shape = self.get_layer_input_shape(self.lrs[0])
        n_input_units = input_shape[-1]
        active_units_indices = list(range(n_input_units))

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        active_units_indices = self.convert_channel_indices_to_flattened_indices(active_units_indices, convolutional_shape)
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                active_units_indices = layer.prune(params.pruning_threshold, active_units_indices)
                last_custom_layer = layer
    
    def grow(self, params):   
        n_new_units = 0

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, CustomLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        n_new_units = n_new_units * convolutional_shape[0] * convolutional_shape[1]
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                n_new_units = layer.grow(n_new_units, params.growth_percentage, min_new_units=params.min_new_neurons, scaling_factor=params.pruning_threshold)
                last_custom_layer = layer
    
    @staticmethod
    def convert_channel_indices_to_flattened_indices(channel_indices, convolutional_shape):
        dense_indices = list()
        units_per_channel = convolutional_shape[0] * convolutional_shape[1]
        for channel_index in channel_indices:
            for iter in range(units_per_channel):
                dense_indices.append(channel_index * units_per_channel + iter)
        return dense_indices
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(layer.get_param_string())
    
    def evaluate(self, params, summed_training_loss, summed_training_accuracy):
        # Calculate training loss and accuracy
        if summed_training_loss is not None:
            loss = summed_training_loss / params.x.shape[0]
        else:
            loss = None
        
        if summed_training_accuracy is not None:
            accuracy = summed_training_accuracy / params.x.shape[0]
        else:
            accuracy = None
        
        # Calculate val loss and accuracy
        summed_val_loss = 0
        summed_val_accuracy = 0
        n_val_instances = 0
        
        for step, (x_batch, y_batch) in enumerate(params.val_dataset):
            y_pred = self(x_batch, training=False)
            summed_val_loss += tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
            summed_val_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))
            n_val_instances += x_batch.shape[0]
        
        val_loss = summed_val_loss / n_val_instances
        val_accuracy = summed_val_accuracy / n_val_instances

        return loss, accuracy, val_loss, val_accuracy
    
    def print_epoch_statistics(self, params, summed_training_loss, summed_training_accuracy, message=None, require_result=False):
        if not params.verbose:
            if require_result:
                return self.evaluate(params, summed_training_loss, summed_training_accuracy)
            else:
                return
        
        loss, accuracy, val_loss, val_accuracy = self.evaluate(params, summed_training_loss, summed_training_accuracy)  

        if message is not None:
            print(message)
        
        print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {self.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total units: {sum(hidden_layer_sizes)}")
        if params.print_neurons:
            self.print_neurons()
        
        if require_result:
            return loss, accuracy, val_loss, val_accuracy
    
    def update_history(self, params, loss, accuracy, val_loss, val_accuracy):
        params.history['loss'].append(loss)
        params.history['accuracy'].append(accuracy)
        params.history['val_loss'].append(val_loss)
        params.history['val_accuracy'].append(val_accuracy)
        params.history['hidden_layer_sizes'].append(self.get_hidden_layer_sizes())
    
    @staticmethod
    def prepare_datasets(x, y, batch_size, validation_data):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=20000).batch(batch_size)
        val_dataset = tf.data.Dataset.from_tensor_slices(validation_data).batch(batch_size)
        return train_dataset.prefetch(tf.data.AUTOTUNE), val_dataset.prefetch(tf.data.AUTOTUNE)
    
    def manage_dynamic_regularization(self, params, val_loss):
        if val_loss >= params.best_conditional_val_loss * params.stall_coefficient:
            # Training is currently in stall
            if not params.training_stalled:
                penalty = self.get_regularization_penalty() * params.regularization_penalty_multiplier
                print("Changing penalty...")
                # TODO this must be modified, penalty can differ for each layer
                self.set_regularization_penalty(penalty)
                params.training_stalled = True
        else:
            params.best_conditional_val_loss = val_loss
            params.training_stalled = False
    
    def grow_wrapper(self, params):
        dynamic_reqularization_active = params.regularization_penalty_multiplier != 1.
        if dynamic_reqularization_active:
            loss, accuracy, val_loss, val_accuracy = self.print_epoch_statistics(params, None, None, "Before growing:", require_result=True)
            self.manage_dynamic_regularization(params, val_loss)
        else:
            self.print_epoch_statistics(params, None, None, "Before growing:")

        self.grow(params)
        self.print_epoch_statistics(params, None, None, "After growing:")
    
    def prune_wrapper(self, params, summed_loss, summed_accuracy):
        loss, accuracy, _, _ = self.print_epoch_statistics(params, summed_loss, summed_accuracy, "Before pruning:", require_result=True)
        self.prune(params)
        _, _, val_loss, val_accuracy = self.print_epoch_statistics(params, None, None, "After pruning:", require_result=True)

        self.update_history(params, loss, accuracy, val_loss, val_accuracy)
    
    class ParameterContainer:
        def __init__(self, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold, 
                regularization_penalty_multiplier, stall_coefficient, growth_percentage, mini_epochs_per_epoch, verbose, print_neurons, use_static_graph):
            self.x = x
            self.y = y
            self.optimizer = optimizer
            self.epochs = epochs
            self.self_scaling_epochs = self_scaling_epochs
            self.batch_size = batch_size
            self.min_new_neurons = min_new_neurons
            self.validation_data = validation_data
            self.pruning_threshold = pruning_threshold
            self.regularization_penalty_multiplier = regularization_penalty_multiplier
            self.stall_coefficient = stall_coefficient
            self.growth_percentage = growth_percentage
            self.mini_epochs_per_epoch = mini_epochs_per_epoch
            self.verbose = verbose
            self.print_neurons = print_neurons
            self.use_static_graph = use_static_graph

            self.train_dataset, self.val_dataset = Sequential.prepare_datasets(x, y, batch_size, validation_data)
            self.history = self.prepare_history()

            self.best_conditional_val_loss = np.inf
            self.training_stalled = False
        
        @staticmethod
        def prepare_history():
            history = {
                'loss': list(),
                'accuracy': list(),
                'val_loss': list(),
                'val_accuracy': list(),
                'hidden_layer_sizes': list(),
            }
            return history
    
    def fit_single_step(self, params, x_batch, y_batch):
        with tf.GradientTape() as tape:
            y_pred = self(x_batch, training=True)
            raw_loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred)
            loss_value = tf.reduce_mean(raw_loss)
            loss_value += sum(self.losses)  # Add losses registered by model.add_loss

            loss = tf.reduce_sum(raw_loss)
            accuracy = float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))

        grads = tape.gradient(loss_value, self.trainable_variables)
        params.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return loss, accuracy
    
    def fit_single_epoch(self, params):
        summed_loss = 0
        summed_accuracy = 0
        
        for mini_epoch in range(params.mini_epochs_per_epoch):
            summed_loss = 0
            summed_accuracy = 0

            if params.use_static_graph:
                fit_single_step_function = tf.function(self.fit_single_step)
            else:
                fit_single_step_function = self.fit_single_step
            for step, (x_batch, y_batch) in enumerate(params.train_dataset):
                loss, accuracy = fit_single_step_function(params, x_batch, y_batch)
                summed_loss += loss
                summed_accuracy += accuracy
        
        return summed_loss, summed_accuracy

    def fit(self, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, regularization_penalty_multiplier=1., 
            stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, pruning_only_epochs=0, verbose=True, print_neurons=False, use_static_graph=True):
        params = self.ParameterContainer(x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, 
                                         validation_data, pruning_threshold, regularization_penalty_multiplier, stall_coefficient, 
                                         growth_percentage, mini_epochs_per_epoch, verbose, print_neurons, use_static_graph)
        self.build(x.shape)  # Necessary when verbose == False

        for epoch in range(epochs):
            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch + 1}/{epochs}")

            if epoch < self_scaling_epochs - pruning_only_epochs:
                self.grow_wrapper(params)
            
            if epoch == self_scaling_epochs:
                self.remove_regularization()

            summed_loss, summed_accuracy = self.fit_single_epoch(params)
            
            if epoch < self_scaling_epochs:
                self.prune_wrapper(params, summed_loss, summed_accuracy)
            else:
                loss, accuracy, val_loss, val_accuracy = self.print_epoch_statistics(params, summed_loss, summed_accuracy, require_result=True)
                self.update_history(params, loss, accuracy, val_loss, val_accuracy)

        return params.history


################################################################################
# HELPER FUNCTIONS
################################################################################


def get_statistics_from_history(history):
    best_epoch_number = np.argmax(history['val_accuracy'])
    best_val_accuracy = history['val_accuracy'][best_epoch_number]
    best_hidden_layer_sizes = history['hidden_layer_sizes'][best_epoch_number]
    return best_val_accuracy, best_hidden_layer_sizes


def get_statistics_from_histories(histories):
    best_val_accuracies = list()
    all_best_hidden_layer_sizes = list()

    for history in histories:
        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        best_val_accuracies.append(best_val_accuracy)
        all_best_hidden_layer_sizes.append(best_hidden_layer_sizes)
    
    mean_best_val_accuracy = np.mean(best_val_accuracies)
    mean_best_hidden_layer_sizes = [np.mean(layer) for layer in list(zip(*all_best_hidden_layer_sizes))]
    
    return mean_best_val_accuracy, mean_best_hidden_layer_sizes


def cross_validate(train_fn, x, y, n_splits, random_state=42, *args, **kwargs):
    from sklearn.model_selection import KFold

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    histories = list()
    for i, (train_index, test_index) in enumerate(kf.split(x)):
        xtrain, xtest = x[train_index], x[test_index]
        ytrain, ytest = y[train_index], y[test_index]

        history = train_fn(xtrain, ytrain, validation_data=(xtest, ytest), *args, **kwargs)
        histories.append(history)

        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run {i} completed, best_val_accuracy: {best_val_accuracy}, best_hidden_layer_sizes: {best_hidden_layer_sizes}")

    mean_best_val_accuracy, mean_best_hidden_layer_sizes = get_statistics_from_histories(histories)
    print(f'mean_best_val_accuracy: {mean_best_val_accuracy}')
    print(f'mean_best_hidden_layer_sizes: {mean_best_hidden_layer_sizes}')

    return histories


def hyperparameter_search(train_fn, x, y, validation_data, *args, **kwargs):
    from itertools import product

    all_params = [*args] + list(kwargs.values())
    histories = list()

    best_overall_val_accuracy = -np.inf
    best_overall_combination = None

    for combination in product(*all_params):
        combination_args = combination[:len(args)]

        combination_kwargs_values = combination[len(args):]
        combination_kwargs = dict(zip(kwargs.keys(), combination_kwargs_values))

        history = train_fn(x, y, validation_data, *combination_args, **combination_kwargs)
        history['parameters'] = combination
        histories.append(history)

        best_val_accuracy, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run with parameters {combination} completed, best_val_accuracy: {best_val_accuracy}, best_hidden_layer_sizes sizes: {best_hidden_layer_sizes}")

        if best_val_accuracy > best_overall_val_accuracy:
            best_overall_val_accuracy = best_val_accuracy
            best_overall_combination = combination
    
    print(f'Best overall combination: {best_overall_combination}, val_accuracy: {best_overall_val_accuracy}')

    return histories

# Accuracy benchmark - FF and convolutions

In [17]:
def get_convolutional_model(x, regularization_penalty, regularization_method, layer_sizes, output_neurons=10):
    model = Sequential([
        Conv2D(layer_sizes[0], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=regularization_penalty, regularization_method=regularization_method, 
            kernel_initializer='lecun_normal', input_shape=x[0,:,:,:].shape),
        Conv2D(layer_sizes[1], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=regularization_penalty, regularization_method=regularization_method, 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(layer_sizes[2], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', 
            regularization_penalty=regularization_penalty, regularization_method=regularization_method, 
            kernel_initializer='lecun_normal'),
        Conv2D(layer_sizes[3], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', 
            regularization_penalty=regularization_penalty, regularization_method=regularization_method, 
            kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        Flatten(),
        Dense(layer_sizes[4], activation='selu', regularization_penalty=regularization_penalty, 
            regularization_method=regularization_method, kernel_initializer='lecun_normal'),
        Dense(output_neurons, activation='softmax', regularization_penalty=0., 
            regularization_method=None, fixed_size=True),
    ])
    return model


def train_fn(x, y, validation_data, learning_rate, regularization_penalty, regularization_method, self_scaling_epochs, layer_sizes, output_neurons=10, 
             epochs=40, pruning_only_epochs=0, min_new_neurons=20, growth_percentage=0.2, verbose=False, use_static_graph=True):
    batch_size = 128

    model = get_convolutional_model(x, regularization_penalty, regularization_method, layer_sizes, output_neurons)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    history = model.fit(x, y, optimizer, epochs, self_scaling_epochs, batch_size, 
                        min_new_neurons, validation_data=validation_data, pruning_only_epochs=pruning_only_epochs, 
                        growth_percentage=growth_percentage, verbose=verbose, use_static_graph=use_static_graph)
    
    return history

## CIFAR10

In [18]:
cifar10 = get_cifar_10_dataset()

### Misc experiments

In [19]:
%%time

histories = hyperparameter_search(train_fn, x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test), 
                                  learning_rate=[0.00005, 0.0002, 0.0006], regularization_penalty=[0.00002], regularization_method=['weighted_l1'], 
                                  self_scaling_epochs=[20], layer_sizes=[[100, 100, 100, 100, 100]])

Run with parameters (5e-05, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.6983, best_hidden_layer_sizes sizes: [100, 44, 39, 55, 63]
Run with parameters (0.0002, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.7588, best_hidden_layer_sizes sizes: [52, 19, 26, 47, 96]
Run with parameters (0.0006, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.768, best_hidden_layer_sizes sizes: [34, 20, 30, 63, 293]
Best overall combination: (0.0006, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]), val_accuracy: 0.768
CPU times: user 9min 21s, sys: 19 s, total: 9min 40s
Wall time: 8min 38s


In [20]:
%%time

histories = cross_validate(train_fn, x=cifar10.X_norm, y=cifar10.y, n_splits=6, learning_rate=0.0006,
                           regularization_penalty=0.00002, regularization_method='weighted_l1',
                           self_scaling_epochs=20, layer_sizes=[100, 100, 100, 100, 100])

Run 0 completed, best_val_accuracy: 0.7676, best_hidden_layer_sizes: [36, 18, 41, 54, 286]
Run 1 completed, best_val_accuracy: 0.7569, best_hidden_layer_sizes: [36, 19, 33, 91, 250]
Run 2 completed, best_val_accuracy: 0.7683, best_hidden_layer_sizes: [34, 18, 26, 67, 261]
Run 3 completed, best_val_accuracy: 0.7627, best_hidden_layer_sizes: [36, 15, 23, 71, 227]
Run 4 completed, best_val_accuracy: 0.7792, best_hidden_layer_sizes: [32, 15, 37, 59, 272]
Run 5 completed, best_val_accuracy: 0.767, best_hidden_layer_sizes: [32, 16, 22, 61, 267]
mean_best_val_accuracy: 0.76695
mean_best_hidden_layer_sizes: [34.333333333333336, 16.833333333333332, 30.333333333333332, 67.16666666666667, 260.5]
CPU times: user 18min 42s, sys: 38 s, total: 19min 20s
Wall time: 16min 25s


In [22]:
%%time

histories = hyperparameter_search(train_fn, x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test), 
                                  learning_rate=[0.0005, 0.002, 0.006], regularization_penalty=[0.], regularization_method=[None], 
                                  self_scaling_epochs=[0], layer_sizes=[[34, 17, 30, 67, 261]])

Run with parameters (0.0005, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.6542, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.002, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.7267, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.006, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.5603, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Best overall combination: (0.002, 0.0, None, 0, [34, 17, 30, 67, 261]), val_accuracy: 0.7267
CPU times: user 6min 43s, sys: 16.9 s, total: 7min
Wall time: 5min 20s


In [23]:
%%time

histories = hyperparameter_search(train_fn, x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test), 
                                  learning_rate=[0.00005, 0.0002, 0.0006], regularization_penalty=[0.], regularization_method=[None], 
                                  self_scaling_epochs=[0], layer_sizes=[[34, 17, 30, 67, 261]])

Run with parameters (5e-05, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.6221, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.0002, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.658, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.0006, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.6556, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Best overall combination: (0.0002, 0.0, None, 0, [34, 17, 30, 67, 261]), val_accuracy: 0.658
CPU times: user 6min 43s, sys: 16.9 s, total: 7min
Wall time: 5min 21s


In [24]:
%%time

histories = hyperparameter_search(train_fn, x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test), 
                                  learning_rate=[0.001, 0.003, 0.004], regularization_penalty=[0.], regularization_method=[None], 
                                  self_scaling_epochs=[0], layer_sizes=[[34, 17, 30, 67, 261]])

Run with parameters (0.001, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.6858, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.003, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.7017, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Run with parameters (0.004, 0.0, None, 0, [34, 17, 30, 67, 261]) completed, best_val_accuracy: 0.6586, best_hidden_layer_sizes sizes: [34, 17, 30, 67, 261]
Best overall combination: (0.003, 0.0, None, 0, [34, 17, 30, 67, 261]), val_accuracy: 0.7017
CPU times: user 6min 42s, sys: 16.7 s, total: 6min 58s
Wall time: 5min 19s


### Reordering evaluation 

In [14]:
batch_size = 128

model = get_convolutional_model(cifar10.X_train_norm, regularization_penalty=0.00002, regularization_method='weighted_l1', layer_sizes=[100, 100, 100, 100, 100], output_neurons=10)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0006)

history = model.fit(cifar10.X_train_norm, cifar10.y_train, optimizer, epochs=20, self_scaling_epochs=20, batch_size=batch_size, 
                    min_new_neurons=20, validation_data=(cifar10.X_test_norm, cifar10.y_test), pruning_only_epochs=0, 
                    growth_percentage=0.2, verbose=True)

##########################################################
Epoch 1/20
Before growing:
loss: None - accuracy: None - val_loss: 3.057939291000366 - val_accuracy: 0.0943 - penalty: 2e-05
hidden layer sizes: [100, 100, 100, 100, 100], total units: 500
After growing:
loss: None - accuracy: None - val_loss: 3.057939291000366 - val_accuracy: 0.0943 - penalty: 2e-05
hidden layer sizes: [120, 120, 120, 120, 120], total units: 600
Before pruning:
loss: 1.7466492652893066 - accuracy: 0.3958800137042999 - val_loss: 1.5053508281707764 - val_accuracy: 0.4652 - penalty: 2e-05
hidden layer sizes: [120, 120, 120, 120, 120], total units: 600
After pruning:
loss: None - accuracy: None - val_loss: 1.505497932434082 - val_accuracy: 0.4655 - penalty: 2e-05
hidden layer sizes: [100, 100, 100, 100, 105], total units: 505
##########################################################
Epoch 2/20
Before growing:
loss: None - accuracy: None - val_loss: 1.505497932434082 - val_accuracy: 0.4655 - penalty: 2e-05
hidden 

In [26]:
batch_size = 128

model = get_convolutional_model(regularization_penalty=0.00002, regularization_method='weighted_l1_reordered', layer_sizes=[100, 100, 100, 100, 100], output_neurons=10)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0006)

history = model.fit(X_train_norm, y_train, optimizer, epochs=20, self_scaling_epochs=20, batch_size=batch_size, 
                    min_new_neurons=20, validation_data=(X_test_norm, y_test), pruning_only_epochs=0, 
                    growth_percentage=0.2, verbose=True)

##########################################################
Epoch 1/20
Before growing:
loss: None - accuracy: None - val_loss: 2.8939507007598877 - val_accuracy: 0.1005 - penalty: 2e-05
hidden layer sizes: [100, 100, 100, 100, 100], total units: 500
After growing:
loss: None - accuracy: None - val_loss: 2.8939504623413086 - val_accuracy: 0.1005 - penalty: 2e-05
hidden layer sizes: [120, 120, 120, 120, 120], total units: 600
Before pruning:
loss: 1.7473422288894653 - accuracy: 0.39658 - val_loss: 1.4361164569854736 - val_accuracy: 0.4886 - penalty: 2e-05
hidden layer sizes: [120, 120, 120, 120, 120], total units: 600
After pruning:
loss: None - accuracy: None - val_loss: 1.436206579208374 - val_accuracy: 0.4881 - penalty: 2e-05
hidden layer sizes: [100, 100, 100, 100, 106], total units: 506
##########################################################
Epoch 2/20
Before growing:
loss: None - accuracy: None - val_loss: 1.436206579208374 - val_accuracy: 0.4881 - penalty: 2e-05
hidden layer siz

In [ ]:
%%time

histories = hyperparameter_search(train_fn, learning_rate=[0.00005, 0.0002, 0.0006], 
                                  regularization_penalty=[0.00002], regularization_method=['weighted_l1'], 
                                  self_scaling_epochs=[20], layer_sizes=[[100, 100, 100, 100, 100]])

Run with parameters (5e-05, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.7074, final_hidden_layer sizes: [100, 46, 47, 53, 71]
Run with parameters (0.0002, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.7496, final_hidden_layer sizes: [50, 25, 23, 60, 86]
Run with parameters (0.0006, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]) completed, best_val_accuracy: 0.7554, final_hidden_layer sizes: [36, 15, 30, 81, 266]
Best overall combination: (0.0006, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100]), val_accuracy: 0.7554
CPU times: user 27min 12s, sys: 31.1 s, total: 27min 43s
Wall time: 27min 44s


In [59]:
%%time

histories = cross_validate(train_fn, X_norm, y, n_splits=6, learning_rate=0.0006,
                           regularization_penalty=0.00002, regularization_method='weighted_l1',
                           self_scaling_epochs=20, layer_sizes=[100, 100, 100, 100, 100])

Run 0 completed, best_val_accuracy: 0.7628, best_hidden_layer_sizes: [37, 17, 28, 77, 258]
Run 1 completed, best_val_accuracy: 0.7695, best_hidden_layer_sizes: [34, 21, 33, 75, 314]
Run 2 completed, best_val_accuracy: 0.7704, best_hidden_layer_sizes: [40, 16, 29, 75, 292]
Run 3 completed, best_val_accuracy: 0.7652, best_hidden_layer_sizes: [26, 15, 28, 73, 254]
Run 4 completed, best_val_accuracy: 0.7672, best_hidden_layer_sizes: [39, 16, 34, 85, 292]
Run 5 completed, best_val_accuracy: 0.7641, best_hidden_layer_sizes: [31, 18, 35, 92, 276]
mean_best_val_accuracy: 0.7665333333333333
mean_best_hidden_layer_sizes: [34.5, 17.166666666666668, 31.166666666666668, 79.5, 281.0]
CPU times: user 20min 30s, sys: 44.1 s, total: 21min 14s
Wall time: 17min 48s


In [8]:
%%time

histories = cross_validate(train_fn, X_norm, y, n_splits=6, learning_rate=0.0006,
                           regularization_penalty=0.00002, regularization_method='weighted_l1_reordered',
                           self_scaling_epochs=20, layer_sizes=[100, 100, 100, 100, 100], use_static_graph=False)

Run 0 completed, best_val_accuracy: 0.7568, best_hidden_layer_sizes: [33, 14, 23, 78, 280]
Run 1 completed, best_val_accuracy: 0.7565, best_hidden_layer_sizes: [32, 13, 33, 75, 329]
Run 2 completed, best_val_accuracy: 0.7552, best_hidden_layer_sizes: [36, 14, 31, 83, 336]
Run 3 completed, best_val_accuracy: 0.7551, best_hidden_layer_sizes: [32, 12, 30, 61, 346]
Run 4 completed, best_val_accuracy: 0.7585, best_hidden_layer_sizes: [34, 14, 29, 66, 304]
Run 5 completed, best_val_accuracy: 0.7516, best_hidden_layer_sizes: [31, 15, 28, 87, 294]
mean_best_val_accuracy: 0.7556166666666666
mean_best_hidden_layer_sizes: [33.0, 13.666666666666666, 29.0, 75.0, 314.8333333333333]
CPU times: user 44min 29s, sys: 1min 7s, total: 45min 36s
Wall time: 44min 25s


## CIFAR100

In [15]:
%%time

histories = hyperparameter_search(train_fn, learning_rate=[0.00005, 0.0002, 0.0006], 
                                  regularization_penalty=[0.00002], regularization_method=['weighted_l1_reordered'], 
                                  self_scaling_epochs=[20], layer_sizes=[[100, 100, 100, 100, 100]], output_neurons=[100], use_static_graph=[False])

Run with parameters (5e-05, 2e-05, 'weighted_l1_reordered', 20, [100, 100, 100, 100, 100], 100, False) completed, best_val_accuracy: 0.3867, best_hidden_layer_sizes sizes: [94, 21, 39, 37, 100]
Run with parameters (0.0002, 2e-05, 'weighted_l1_reordered', 20, [100, 100, 100, 100, 100], 100, False) completed, best_val_accuracy: 0.4364, best_hidden_layer_sizes sizes: [62, 16, 23, 66, 246]
Run with parameters (0.0006, 2e-05, 'weighted_l1_reordered', 20, [100, 100, 100, 100, 100], 100, False) completed, best_val_accuracy: 0.4187, best_hidden_layer_sizes sizes: [46, 21, 55, 93, 899]
Best overall combination: (0.0002, 2e-05, 'weighted_l1_reordered', 20, [100, 100, 100, 100, 100], 100, False), val_accuracy: 0.4364
CPU times: user 21min 43s, sys: 25.1 s, total: 22min 8s
Wall time: 20min 47s


In [14]:
%%time

histories = cross_validate(train_fn, X_norm, y, n_splits=6, learning_rate=0.0002,
                           regularization_penalty=0.00002, regularization_method='weighted_l1_reordered',
                           self_scaling_epochs=20, layer_sizes=[100, 100, 100, 100, 100], output_neurons=100, use_static_graph=False)

Run 0 completed, best_val_accuracy: 0.4363, best_hidden_layer_sizes: [62, 15, 22, 65, 256]
Run 1 completed, best_val_accuracy: 0.453, best_hidden_layer_sizes: [71, 17, 22, 51, 265]
Run 2 completed, best_val_accuracy: 0.4458, best_hidden_layer_sizes: [62, 16, 27, 57, 277]
Run 3 completed, best_val_accuracy: 0.4425, best_hidden_layer_sizes: [63, 19, 22, 82, 261]
Run 4 completed, best_val_accuracy: 0.4468, best_hidden_layer_sizes: [61, 19, 21, 79, 249]
Run 5 completed, best_val_accuracy: 0.4441, best_hidden_layer_sizes: [68, 14, 20, 77, 274]
mean_best_val_accuracy: 0.44475000000000003
mean_best_hidden_layer_sizes: [64.5, 16.666666666666668, 22.333333333333332, 68.5, 263.6666666666667]
CPU times: user 44min 10s, sys: 3min 14s, total: 47min 25s
Wall time: 44min 35s


In [ ]:
%%time

histories = hyperparameter_search(train_fn, learning_rate=[0.00005, 0.0002, 0.0006], 
                                  regularization_penalty=[0.00002], regularization_method=['weighted_l1'], 
                                  self_scaling_epochs=[20], layer_sizes=[[100, 100, 100, 100, 100]], output_neurons=[100])

Run with parameters (5e-05, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100], 100) completed, best_val_accuracy: 0.4097, final_hidden_layer sizes: [100, 40, 48, 48, 100]
Run with parameters (0.0002, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100], 100) completed, best_val_accuracy: 0.451, final_hidden_layer sizes: [69, 21, 33, 67, 193]
Run with parameters (0.0006, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100], 100) completed, best_val_accuracy: 0.4063, final_hidden_layer sizes: [52, 21, 100, 124, 808]
Best overall combination: (0.0002, 2e-05, 'weighted_l1', 20, [100, 100, 100, 100, 100], 100), val_accuracy: 0.451
CPU times: user 18min 25s, sys: 19.1 s, total: 18min 45s
Wall time: 20min 34s


In [ ]:
%%time

histories = cross_validate(train_fn, X_norm, y, n_splits=6, learning_rate=0.0002,
                           regularization_penalty=0.00002, regularization_method='weighted_l1',
                           self_scaling_epochs=20, layer_sizes=[100, 100, 100, 100, 100], output_neurons=100)

Run 0 completed, best_val_accuracy: 0.4509, final_hidden_layer sizes: [65, 22, 30, 68, 200]
Run 1 completed, best_val_accuracy: 0.4497, final_hidden_layer sizes: [63, 24, 31, 69, 206]
Run 2 completed, best_val_accuracy: 0.4431, final_hidden_layer sizes: [63, 23, 28, 81, 189]
Run 3 completed, best_val_accuracy: 0.4468, final_hidden_layer sizes: [65, 27, 32, 70, 206]
Run 4 completed, best_val_accuracy: 0.459, final_hidden_layer sizes: [68, 19, 30, 69, 200]
Run 5 completed, best_val_accuracy: 0.4536, final_hidden_layer sizes: [65, 24, 25, 75, 204]
mean_best_val_accuracy: 0.4505166666666667
mean_final_hidden_layer_sizes: [64.83333333333333, 23.166666666666668, 29.333333333333332, 72.0, 200.83333333333334]
CPU times: user 36min 16s, sys: 35.5 s, total: 36min 52s
Wall time: 39min 4s
